In [ ]:
import os
import io
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

# Scopes for Google Drive access
SCOPES = ['https://www.googleapis.com/auth/drive']

# Authenticate using OAuth 2.0
def authenticate():
    flow = InstalledAppFlow.from_client_secrets_file(
        'client_secrets.json', SCOPES)
    creds = flow.run_local_server(port=0)
    return build('drive', 'v3', credentials=creds)

# List TIFF files in a shared folder
def list_tiff_files(service, folder_id):
    query = f"'{folder_id}' in parents and mimeType='image/tiff'"
    results = service.files().list(q=query, fields="files(id, name)").execute()
    return results.get('files', [])

# Download a file
def download_file(service, file_id, file_name, output_dir):
    request = service.files().get_media(fileId=file_id)
    file_path = os.path.join(output_dir, file_name)
    with io.FileIO(file_path, 'wb') as file:
        downloader = MediaIoBaseDownload(file, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Downloading {file_name}: {int(status.progress() * 100)}%")
    print(f"Downloaded {file_name} to {file_path}")

def main():
    # Authenticate and create the service
    service = authenticate()

    # Specify the shared folder ID
    SHARED_FOLDER_ID = '1Vxvb8lStfp3s_NOs8xy9r6NP2TV5uT6Z'

    # Output directory for downloaded files
    output_dir = 'downloaded_tiff_images'
    os.makedirs(output_dir, exist_ok=True)

    # List and download TIFF files
    tiff_files = list_tiff_files(service, SHARED_FOLDER_ID)
    if not tiff_files:
        print("No TIFF files found.")
        return

    for file in tiff_files:
        download_file(service, file['id'], file['name'], output_dir)

if __name__ == '__main__':
    main()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1007301603530-of60vh7oh54n3ruao261tms36kehruan.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A62935%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=T95EhukNPFKuz2RJnAVTtTCWSmzonk&access_type=offline


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

# Authenticate with PyDrive
gauth = GoogleAuth()
gauth.LocalWebserverAuth()  # Opens a local server for authentication
drive = GoogleDrive(gauth)

# List files in a folder
folder_id = '1jaLtYUp94shvmp2H0RPpOHevdueJnkfE'
file_list = drive.ListFile({'q': f"'{folder_id}' in parents"}).GetList()

# Download TIFF files
output_dir = 'downloaded_tiff_images'
os.makedirs(output_dir, exist_ok=True)

for file in file_list:
    if file['mimeType'] == 'image/tif':
        print(f"Downloading {file['title']}...")
        file.GetContentFile(f"{output_dir}/{file['title']}")


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1007301603530-of60vh7oh54n3ruao261tms36kehruan.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8090%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

